## Train

In [ ]:
%%capture
!pip install wandb transformers ml_collections

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import json
import tqdm
from IPython.display import clear_output, Image, display
import PIL.Image
import io
import json
import torch
import numpy as np
import pickle
import os
from torch.utils.data import Dataset, DataLoader
import json
import os
from glob import glob
from torchvision import transforms as T
from PIL import Image

from torch import nn
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
import random
import numpy as np


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
label_mapping = ["flooded",
"non flooded",
"flooded,non flooded",
"Yes",
"No",
"0",
"1",
"2",
"3",
"4",
"5",
"6",
"7",
"8",
"9",
"10",
"11",
"12",
"13",
"14",
"15",
"16",
"17",
"18",
"19",
"20",
"21",
"22",
"23",
"24",
"25",
"26",
"27",
"28",
"29",
"30",
"31",
"32",
"33",
"34",
"35",
"36",
"37",
"38",
"39",
"40",
"41",
"42",
"43",
"44",
"45",
"46",
"47",
"48",
"49",
"50"]

In [ ]:
qdict = json.load(open("/content/drive/MyDrive/floodnet_data/Questions/Training Question.json", "r"))

def get_uniq_image_ids(jsondict):
    uniq_images = []
    for key, example in jsondict.items():
        if example["Image_ID"] not in uniq_images:
            uniq_images.append(example["Image_ID"])
    return uniq_images

def get_questions_for_imageid(jsondict, imageid):
    qs = dict()
    for key, example in jsondict.items():
        if example["Image_ID"] == imageid:
            qs[key] = example
    return qs

def get_train_val_splits_imagewise(jsondict):
    train_dict = dict()
    val_dict = dict()

    uniq_images = get_uniq_image_ids(jsondict)
    
    train, val = train_test_split(uniq_images, test_size=0.2)

    for imageid in train:
        train_dict.update(get_questions_for_imageid(jsondict, imageid)) 

    for imageid in val:
        val_dict.update(get_questions_for_imageid(jsondict, imageid))

    return train_dict, val_dict
# train_dict, 
def get_uniq_image_ids(jsondict):
    uniq_images = []
    for key, example in jsondict.items():
        if example["Image_ID"] not in uniq_images:
            uniq_images.append(example["Image_ID"])
    return uniq_images

def get_questions_for_imageid(jsondict, imageid):
    qs = dict()
    for key, example in jsondict.items():
        if example["Image_ID"] == imageid:
            qs[key] = example
    return qs

def get_train_val_splits_imagewise(jsondict):
    train_dict = dict()
    val_dict = dict()

    uniq_images = get_uniq_image_ids(jsondict)
    
    train, val = train_test_split(uniq_images, test_size=0.2)

    for imageid in train:
        train_dict.update(get_questions_for_imageid(jsondict, imageid)) 

    for imageid in val:
        val_dict.update(get_questions_for_imageid(jsondict, imageid))

    return train_dict, val_dict
    
train_dict, val_dict,  = get_train_val_splits_imagewise(qdict)

In [ ]:
from transformers import ViltProcessor, ViltModel, ViltForQuestionAnswering

class FloodNetVQADataset(Dataset):
    def __init__(self, images_root_path, jsondict):
        super().__init__()

        self.json_dict = jsondict
        self.images_root_path = images_root_path

        self.device = torch.device(
            "cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.transforms = T.Compose([
            T.ToTensor(),
            T.Resize((512,512))            
        ])
        self.processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

        self.reset_index()

    def reset_index(self):
        new_qdict = dict()
        for idx, value in enumerate(self.json_dict.values()):
            new_qdict[str(idx)] = value
        self.json_dict = new_qdict


    def __len__(self):
        return len(self.json_dict.keys())

    def __getitem__(self, idx):
        # Image, Question, Answer
        example = self.json_dict[str(idx)]
        imagepath = os.path.join(self.images_root_path, example["Image_ID"])
        # print(example["Question"])
        question = example["Question"]
        answer = torch.tensor(label_mapping.index(str(example["Ground_Truth"])))

        # print(question["input_ids"].shape)
        # print(question["token_type_ids"].shape)
        # print(question["attention_mask"].shape)

        img = self.transforms(Image.open(imagepath))

        inputs = self.processor(img, question,
                                add_special_tokens=True,
                                return_attention_mask=True,
                                max_length=40,
                                padding='max_length',
                                truncation=True,
                                return_tensors="pt")
        
        return inputs, answer

In [ ]:
# from transformers import ViltProcessor, ViltModel
# from PIL import Image
# import requests

# # prepare image and text
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
# text = "hello world"

# processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
# model = ViltModel.from_pretrained("dandelin/vilt-b32-mlm")

# inputs = processor(image, text, return_tensors="pt")
# outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state

In [ ]:
# model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
# # (0): Linear(in_features=768, out_features=1536, bias=True)
# #     (1): LayerNorm((1536,), eps=1e-05, elementwise_affine=Tru
# model.classifier.append(nn.LayerNorm((3129,), eps=1e-05, elementwise_affine=True))
# model.classifier.append(nn.GELU())
# model.classifier.append(nn.Linear(3129, 56))

In [ ]:
# model[:-1]

In [ ]:

class VQAModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
        self.model.classifier.append(nn.LayerNorm((3129,), eps=1e-05, elementwise_affine=True))
        self.model.classifier.append(nn.GELU())
        self.model.classifier.append(nn.Linear(3129, len(label_mapping)))
        for param in self.model.parameters():
            param.requires_grad = False
        for param in self.model.classifier.parameters():
            param.requires_grad = True
        # self.linear = nn.Linear(self.model.config.hidden_size, len(label_mapping))
    
    def forward(self, batch):
        # outputs = self.model(**batch[0])[1]
        outputs = self.model(**batch[0])
        # print(len(outputs), type(outputs[0]))
        # x = self.linear(outputs)
        return outputs


In [ ]:
train_ds = FloodNetVQADataset(
    images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image",
    jsondict=train_dict
)

val_ds = FloodNetVQADataset(
    images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image",
    jsondict=val_dict
)

# train_dl = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=8)
# val_dl = DataLoader(val_ds, batch_size=8, shuffle=True, num_workers=8)

In [ ]:
# for i in train_dl:
#     print(i)
#     break

In [ ]:
# i

In [ ]:
# batch = next(iter(train_ds))

In [ ]:
# batch[1]

In [ ]:
# batch[0].shape

In [ ]:
model = VQAModel()
# model(batch[0]).logits

Downloading:   0%|          | 0.00/132k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449M [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

In [ ]:
# train_ds["3620"] = train_ds["3619"] # error --> to fix this use below: ``` if num == len(train_ds): pass ```

TypeError: ignored

In [ ]:
adfad

In [ ]:
model = VQAModel()

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

model.train()
model.to(device)
for epoch in range(5):
    print(f"Epoch: {epoch}")
    for num, batch in enumerate(tqdm(train_ds)):
        inp, ans = batch
        inp, ans = inp.to(device), ans.to(device)
        ans = ans.unsqueeze(0)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model((inp, ans)).logits

        loss = criterion(outputs, ans)
        loss.backward()
        optimizer.step()
        if num%100 == 0:
          print(loss)
        if num==len(train_ds)-1:
          break

    torch.save(model.state_dict(), f"/content/drive/MyDrive/{epoch}.pt")

Epoch: 0


  0%|          | 1/3620 [00:00<24:52,  2.42it/s]

tensor(4.4767, device='cuda:0', grad_fn=<NllLossBackward0>)


  3%|▎         | 101/3620 [00:41<23:32,  2.49it/s]

tensor(0.0961, device='cuda:0', grad_fn=<NllLossBackward0>)


...

 97%|█████████▋| 3501/3620 [23:21<00:49,  2.40it/s]

tensor(0.0184, device='cuda:0', grad_fn=<NllLossBackward0>)


 99%|█████████▉| 3601/3620 [24:00<00:07,  2.60it/s]

tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|█████████▉| 3619/3620 [24:07<00:00,  2.50it/s]


In [ ]:
!nvidia-smi # Tesla T4, 15 GB, 

Sun May 29 11:30:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |   2914MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!# model
!mkdir /content/drive/MyDrive/floodnet_features/models

In [ ]:
PATH = "/content/drive/MyDrive/floodnet_features/models/model_weights.pth"
# torch.save(model.state_dict(), PATH)

In [ ]:
!ls /content/drive/MyDrive

 0.pt   3.pt	      'Colab Notebooks'    Neeraja      WASSA
 1.pt   4.pt	       floodnet_data	   Privacy_ML
 2.pt   Anjali_Sahil   floodnet_features   rfmid_ws


## Evaluation

In [ ]:
train_ds = FloodNetVQADataset(
    images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image",
    jsondict=train_dict
)

val_ds = FloodNetVQADataset(
    images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image",
    jsondict=val_dict
)

simple_count = dict()
complex_count = dict()
yes_no = dict()
entire_image = dict()
road_condition = dict()

counter = 0
for example in val_dict.values():
    if example["Question_Type"] == "Yes_No":
        yes_no[str(counter)] = example
        counter += 1
    elif example["Question_Type"] == "Simple_Counting":
        simple_count[str(counter)] = example
        counter += 1
    elif example["Question_Type"] == "Complex_Counting":
        complex_count[str(counter)] = example
        counter += 1
    elif "road" in example["Question"]:
        road_condition[str(counter)] = example
        counter += 1
    elif "overall" in example["Question"]:
        entire_image[str(counter)] = example
        counter += 1

train_ds = FloodNetVQADataset(
    images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image",
    jsondict=train_dict
)

simple_count = FloodNetVQADataset(images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image", jsondict=simple_count)
complex_count = FloodNetVQADataset(images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image", jsondict=complex_count)
road_condition = FloodNetVQADataset(images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image", jsondict=road_condition)
yes_no = FloodNetVQADataset(images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image", jsondict=yes_no)
entire_image = FloodNetVQADataset(images_root_path="/content/drive/MyDrive/floodnet_data/Images/Train_Image", jsondict=entire_image)

In [ ]:
def evaluate(model, ds):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.eval()
  model.to(device)
  preds = []
  ground_truths = []
  for num, batch in enumerate(tqdm(ds)):
    inp, ans = batch
    inp, ans = inp.to(device), ans.to(device)
    ans = ans.unsqueeze(0)

    outputs = model((inp, ans)).logits
    preds.extend(torch.argmax(outputs, dim=1).cpu())
    ground_truths.extend(ans.cpu())
    if num==len(ds)-1:
      break
  return accuracy_score(ground_truths, preds)

In [ ]:
def evaluate_pretrained(model, ds):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.eval()
  model.to(device)
  preds = []
  ground_truths = []
  for num, batch in enumerate(tqdm(ds)):
    inp, ans = batch
    inp, ans = inp.to(device), ans.to(device)
    ans = ans.unsqueeze(0)
    # print(model(**inp).logits)
    pred = model.config.id2label[model(**inp).logits.argmax(-1).item()]

    preds.append(pred)
    ground_truths.append(str(ans.item()))
    if num==len(ds)-1:
      break
  print(ground_truths, preds, sep='\n')
  return accuracy_score(ground_truths, preds)

### Vilt fine-tuned

In [ ]:
model = VQAModel()
# PATH = "/content/drive/MyDrive/floodnet_features/models/model_weights.pth"
PATH = "/content/drive/MyDrive/1.pt"
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
evaluate(model, val_ds)

100%|█████████▉| 890/891 [05:58<00:00,  2.48it/s]


0.7643097643097643

In [ ]:
# evaluate(model, simple_count)

In [ ]:
evaluate(model, complex_count)

 99%|█████████▉| 130/131 [00:53<00:00,  2.45it/s]


0.20610687022900764

In [ ]:
evaluate(model, road_condition)

 99%|█████████▉| 176/177 [01:10<00:00,  2.51it/s]


0.9661016949152542

In [ ]:
evaluate(model, yes_no)

 99%|█████████▉| 176/177 [01:10<00:00,  2.49it/s]


0.9887005649717514

In [ ]:
evaluate(model, entire_image)

100%|█████████▉| 289/290 [01:55<00:00,  2.50it/s]


0.9655172413793104

In [ ]:
!ls /content/drive/MyDrive/floodnet_features/Questions/H*

'/content/drive/MyDrive/floodnet_features/Questions/How many buildings are flooded in this image?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many buildings are flooded?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many buildings are in the image?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many buildings are in this image?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many buildings are non flooded in this image?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many buildings are non flooded?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many buildings can be seen in the image?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many buildings can be seen in this image?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many flooded buildings can be seen in this image?.pt'
'/content/drive/MyDrive/floodnet_features/Questions/How many non flooded buildings can be seen in this image?.pt

In [ ]:
image = torch.load("/content/drive/MyDrive/floodnet_features/Images/Train_Image/6359.pt")

In [ ]:
image.shape

torch.Size([2048])

In [ ]:
text = torch.load('/content/drive/MyDrive/floodnet_features/Questions/How many buildings are flooded in this image?.pt')
text.shape

torch.Size([1, 1024])

In [ ]:
2048 + 1024

3072

### Vilt pre-trained

In [ ]:
from transformers import ViltForQuestionAnswering

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
# PATH = "/content/drive/MyDrive/floodnet_features/models/model_weights.pth"
# model.load_state_dict(torch.load(PATH))

In [ ]:
evaluate_pretrained(model, val_ds)

 37%|███▋      | 330/891 [06:24<12:03,  1.29s/it]

In [ ]:
evaluate_pretrained(model, simple_count)

 99%|█████████▉| 115/116 [00:46<00:00,  2.49it/s]

['7', '11', '9', '6', '6', '29', '6', '6', '11', '9', '8', '8', '8', '13', '7', '15', '9', '8', '6', '6', '9', '9', '6', '21', '25', '8', '18', '7', '8', '7', '7', '19', '8', '11', '8', '6', '11', '13', '12', '12', '7', '22', '7', '6', '23', '6', '9', '9', '9', '17', '11', '9', '9', '7', '9', '21', '8', '6', '9', '6', '9', '9', '10', '16', '7', '8', '9', '20', '19', '7', '16', '23', '8', '10', '9', '21', '22', '6', '8', '9', '7', '12', '9', '21', '6', '8', '9', '17', '8', '12', '9', '23', '9', '6', '18', '9', '13', '10', '6', '8', '26', '6', '7', '8', '8', '29', '9', '25', '20', '6', '10', '15', '28', '8', '8', '8']
['1', '2', '2', '1', '2', '1', '1', '1', '1', '2', '1', '3', '2', '2', '2', '4', '3', '3', '0', '2', '3', '3', '1', '6', '1', '2', '6', '1', '0', '1', '3', '6', '1', '2', '2', '1', '1', '5', '3', '3', '2', '6', '2', '0', '20', '1', '3', '1', '2', '3', '2', '3', '3', '1', '1', '20', '2', '2', '1', '1', '2', '2', '2', '2', '2', '2', '1', '4', '2', '2', '0', '6', '2', '4', '2'

0.0

In [ ]:
evaluate_pretrained(model, complex_count)

 99%|█████████▉| 130/131 [00:52<00:00,  2.47it/s]

['7', '11', '9', '6', '6', '29', '6', '6', '11', '9', '8', '8', '8', '13', '7', '10', '10', '9', '8', '6', '6', '9', '9', '6', '21', '25', '8', '18', '7', '8', '7', '7', '19', '8', '11', '8', '6', '9', '7', '12', '6', '12', '12', '7', '22', '7', '6', '9', '19', '6', '9', '9', '9', '6', '16', '11', '9', '9', '7', '9', '21', '8', '6', '9', '6', '9', '9', '10', '16', '7', '8', '9', '20', '18', '6', '7', '16', '23', '8', '10', '9', '21', '14', '13', '6', '8', '9', '7', '10', '7', '9', '12', '14', '6', '8', '9', '7', '15', '8', '12', '9', '10', '18', '9', '6', '18', '9', '13', '10', '6', '8', '26', '6', '7', '8', '8', '6', '28', '9', '11', '19', '20', '6', '10', '6', '14', '25', '8', '8', '8', '8']
['1', '2', '2', '1', '1', '0', '1', '1', '1', '1', '1', '2', '1', '2', '1', '2', '1', '2', '2', '0', '1', '2', '2', '1', '2', '1', '2', '3', '1', '0', '1', '1', '3', '1', '1', '2', '1', '1', '1', '2', '2', '3', '2', '2', '2', '1', '1', '1', '3', '1', '2', '1', '2', '2', '2', '2', '2', '2', '1', '

0.0

In [ ]:
evaluate_pretrained(model, road_condition)

 99%|█████████▉| 176/177 [01:10<00:00,  2.50it/s]

['1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '2', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '2', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '0', '1', '0', '0', '1', '1']
['clean', 'poor', 'good', 'clean', 'bad', 'bad', 'clean', 'bad', 'bad', 'bad', 'bad', 'poor', 'bad', 'clean', 'bad

0.0

In [ ]:
evaluate_pretrained(model, yes_no)

 99%|█████████▉| 176/177 [01:10<00:00,  2.50it/s]

['3', '3', '4', '3', '4', '3', '4', '3', '4', '3', '3', '3', '3', '4', '4', '3', '3', '3', '3', '4', '4', '4', '3', '3', '4', '3', '4', '3', '3', '4', '3', '4', '3', '3', '4', '4', '3', '3', '3', '3', '4', '4', '3', '3', '4', '3', '3', '3', '4', '3', '3', '4', '4', '4', '3', '3', '3', '4', '4', '4', '3', '3', '4', '4', '4', '3', '4', '3', '4', '4', '3', '4', '4', '4', '4', '4', '3', '4', '3', '3', '3', '4', '3', '3', '3', '3', '3', '4', '4', '3', '4', '4', '4', '4', '4', '4', '3', '4', '4', '4', '3', '4', '4', '4', '4', '4', '3', '3', '3', '4', '4', '4', '3', '4', '3', '4', '3', '3', '3', '4', '3', '4', '3', '4', '4', '4', '3', '4', '3', '4', '4', '4', '3', '3', '3', '3', '4', '4', '3', '3', '3', '3', '4', '4', '4', '4', '4', '3', '4', '3', '4', '3', '4', '4', '4', '3', '4', '4', '3', '3', '3', '3', '4', '3', '3', '4', '4', '4', '4', '4', '3', '4', '3', '4', '3', '4', '3']
['no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', '

0.0

In [ ]:
evaluate_pretrained(model, entire_image)

100%|█████████▉| 289/290 [01:58<00:00,  2.45it/s]

['1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1',

0.0